In [1]:
import os
import json
import time
import CustomLoader
import loader as Loader
import parser as Parser
from bs4 import BeautifulSoup
from LoginScript import LoginScript
from custom_parser.PageParser import PageParser

In [2]:
# donload pages wiht job offers

loader = Loader.Loader()
base_url = 'https://it.pracuj.pl/praca'
file_names = []
for page_number in range(1, 98):
    page_url = f"{base_url}?pn={page_number}" if page_number > 1 else base_url
    file_names.append( loader.crawler(page_url, f'main_page_page{page_number}') )

In [3]:
# extract job offers urls 
file_names = os.listdir('temp_data')
for file_name in file_names:
    base_file_name = file_name.split('_')[2]
    parser = Parser.Parser( f"./temp_data/{file_name}", base_file_name)
    parser.parse()

In [2]:
# dowload html offer pages
todos = []
for todo_file in os.listdir('todos'):
    with open(f"./todos/{todo_file}") as f:
        [todos.append(line.strip()) for line in f]

cookies = False
loader = Loader.Loader()
for url2todo in todos:
    if not(cookies):
        cookies = LoginScript().click_button_and_get_cookies(url2todo)
        cookies = { ck['name']:ck['value'] for ck in cookies}
        
    result = CustomLoader.CustomLoader.crawl(url2todo,'new_pages/offer', cookies)
    if not(result):
        cookies = LoginScript().click_button_and_get_cookies(url2todo)
        cookies = { ck['name']:ck['value'] for ck in cookies}

        with open('unparsable_todo.txt', 'a') as f:
            f.write(url2todo)

In [ ]:
os.listdir('./offer_data/new_pages')

In [5]:
# extract necessary data from offer pages
file_name = os.listdir('./offer_data/new_pages')
for i in range(0, len(os.listdir('./offer_data/new_pages')), 500):
    page_jsons = []
    last = i+500 if i+500 < len(file_name) else len(file_name)
    for file in file_name[i:last]:
        page_jsons.append(PageParser(f"./offer_data/new_pages/{file}").parse_json())
    with open(f'./offer_data/pl_json_pages/{int( time.time() *1000000 )}.json', 'w', encoding="utf-8") as f:
        json.dump(page_jsons, f)

In [8]:
len(os.listdir('./offer_data/new_pages'))/500

9.632